In [2]:
import os, glob, copy
from datetime import date
from la_forge import core as co
import numpy as np

## PTMCMC noise runs

In [3]:
basedir = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
datadir = f'{basedir}/data/lite_unfiltered_53_ePSRs'
template_file = './template_DR2lite_unfiltered_advnoise.sh'
todays_date = str(date.today()).replace('-','')

In [4]:
# Assume 8 runs and we want 200k samples, thin of 10, assume burn of 5000 (max)
thin = 10
Nchains = 8
burn = 5000
target = 200000
single_chain_Niter = (target//Nchains+burn)*10
single_chain_Niter

300000

In [5]:
# get psrnames
psrlist = []
psrs = glob.glob(datadir + '/J*')
for psr in psrs:
    name = psr.split('/')[-1][:-4]
    psrlist.append(name)
psrlist = np.sort(np.unique(psrlist))
# overwrite list
psrlist = ['J0610-2100', 'J0900-3144', 'J1614-2230', 'J1721-2457', 'J1741+1351',
           'J1824-2452A', 'J1903+0327', 'J1944+0907', 'J2019+2425', 'J2033+1734']

In [6]:
psrlist, len(psrlist)

(['J0610-2100',
  'J0900-3144',
  'J1614-2230',
  'J1721-2457',
  'J1741+1351',
  'J1824-2452A',
  'J1903+0327',
  'J1944+0907',
  'J2019+2425',
  'J2033+1734'],
 10)

In [9]:
with open(template_file, "r") as f:
    template = f.read()

In [10]:
print(template)

#!/bin/bash
#SBATCH --job-name=DR2lite_unfiltered_{{PSRNAME}}_advnoise_{{DATE}}
#SBATCH --partition={{PARTITION}}
#SBATCH --exclude=r806u23n04
#SBATCH --output=/home/bbl29/IPTA_DR2_analysis/logs/{{PSRNAME}}_DR2lite_unfiltered_advnoise_{{DATE}}.txt
#SBATCH --error=/home/bbl29/IPTA_DR2_analysis/logs/error/{{PSRNAME}}_DR2lite_unfiltered_advnoise_{{DATE}}.txt
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=1
#SBATCH --threads-per-core=1
#SBATCH --mem-per-cpu=10G
#SBATCH --time={{RUNTIME_DAYS}}-
#SBATCH --array=0-{{ARRAY_MAX}}
#SBATCH --mail-type=END
#SBATCH --mail-user=bjorn.larsen@yale.edu
#SBATCH --requeue

echo "SLURM_JOBID: " $SLURM_JOBID
echo "SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID
echo "SLURM_ARRAY_JOB_ID: " $SLURM_ARRAY_JOB_ID

module load miniconda
module load OpenMPI
conda activate PTA_env

psrname={{PSRNAME}}
dataset=lite_unfiltered_53
Niter=350000
outdir=/vast/palmer/scratch/mingarelli/bbl29/IPTA_DR2_analysis/chains/dr2lite_unfiltered/advnoise/{{PSRNAME}}
psr={

In [11]:
psrs_to_run_extra_time = []#['J0613-0200','J1012+5307','J1024-0719','J1600-3053',
                          #'J1643-1224','J1713+0747','J1744-1134','J1857+0943',
                          #'J1909-3744','J1939+2134','J2145-0750']

In [12]:
for psrname in psrlist:
    print(psrname)
    # Write new file
    file_path = f'{basedir}/jobs/{psrname}_DR2lite_unfiltered_advnoise.sh'
    job_text = copy.deepcopy(template)
    job_text = job_text.replace("{{PSRNAME}}", psrname)
    job_text = job_text.replace("{{DATE}}", todays_date)
    job_text = job_text.replace("{{PSR}}", f'{datadir}/{psrname}.pkl')
    if psrname in psrs_to_run_extra_time:
        print('run pi_mingarelli')
        job_text = job_text.replace("{{PARTITION}}", 'pi_mingarelli')
        job_text = job_text.replace("{{RUNTIME_DAYS}}", '2')
        job_text = job_text.replace("{{ARRAY_MAX}}", '7')
    else:
        job_text = job_text.replace("{{PARTITION}}", 'scavenge')
        job_text = job_text.replace("{{RUNTIME_DAYS}}", '1')
        job_text = job_text.replace("{{ARRAY_MAX}}", '9')
    with open(file_path, "w") as f:
        f.write(job_text)

J0610-2100
J0900-3144
J1614-2230
J1721-2457
J1741+1351
J1824-2452A
J1903+0327
J1944+0907
J2019+2425
J2033+1734


In [14]:
print(job_text)

#!/bin/bash
#SBATCH --job-name=DR2lite_unfiltered_J2033+1734_advnoise_20250303
#SBATCH --partition=scavenge
#SBATCH --exclude=r806u23n04
#SBATCH --output=/home/bbl29/IPTA_DR2_analysis/logs/J2033+1734_DR2lite_unfiltered_advnoise_20250303.txt
#SBATCH --error=/home/bbl29/IPTA_DR2_analysis/logs/error/J2033+1734_DR2lite_unfiltered_advnoise_20250303.txt
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=1
#SBATCH --threads-per-core=1
#SBATCH --mem-per-cpu=10G
#SBATCH --time=1-
#SBATCH --array=0-9
#SBATCH --mail-type=END
#SBATCH --mail-user=bjorn.larsen@yale.edu
#SBATCH --requeue

echo "SLURM_JOBID: " $SLURM_JOBID
echo "SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID
echo "SLURM_ARRAY_JOB_ID: " $SLURM_ARRAY_JOB_ID

module load miniconda
module load OpenMPI
conda activate PTA_env

psrname=J2033+1734
dataset=lite_unfiltered_53
Niter=350000
outdir=/vast/palmer/scratch/mingarelli/bbl29/IPTA_DR2_analysis/chains/dr2lite_unfiltered/advnoise/J2033+1734
psr=/vast/palmer/home.grace/bbl29/IPTA_DR2

In [15]:
mass_submit_text = 'import os\n'
for psrname in psrlist:
    mass_submit_text += f'os.system(f"sbatch jobs/{psrname}_DR2lite_unfiltered_advnoise.sh")\n'
with open(f'{basedir}/submit_DR2lite_10more.py', "w") as f:
    f.write(mass_submit_text)

## Nautilus fact like runs

In [26]:
datastr = 'lite_unfiltered'
basedir = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
datadir = f'{basedir}/data/{datastr}_53_ePSRs'
#template_file = f'./template_{datastr}DR2_advnoise.sh'
template_file = f'./template_nautilus_FL_dr2{datastr}.sh'
todays_date = str(date.today()).replace('-','')

In [27]:
# get psrnames
psrlist = []
psrs = glob.glob(datadir + '/J*')
for psr in psrs:
    name = psr.split('/')[-1][:-4]
    psrlist.append(name)
psrlist = np.sort(np.unique(psrlist))
# overwrite list
psrlist = ['J0610-2100', 'J0900-3144', 'J1614-2230', 'J1721-2457', 'J1741+1351',
           'J1824-2452A', 'J1903+0327', 'J1944+0907', 'J2019+2425', 'J2033+1734']
psrlist, len(psrlist)

(['J0610-2100',
  'J0900-3144',
  'J1614-2230',
  'J1721-2457',
  'J1741+1351',
  'J1824-2452A',
  'J1903+0327',
  'J1944+0907',
  'J2019+2425',
  'J2033+1734'],
 10)

In [28]:
with open(template_file, "r") as f:
    template = f.read()

In [29]:
print(template)

#!/bin/bash
#SBATCH --job-name=DR2lite_unfiltered_{{PSRNAME}}_factlike_{{DATE}}
#SBATCH --partition=scavenge
#SBATCH --exclude=r806u23n04
#SBATCH --output=/home/bbl29/IPTA_DR2_analysis/logs/DR2lite_unfiltered_{{PSRNAME}}_factlike_{{DATE}}.txt
#SBATCH --error=/home/bbl29/IPTA_DR2_analysis/logs/error/DR2lite_unfiltered_{{PSRNAME}}_factlike_{{DATE}}.txt
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=10G
#SBATCH --time=2:00:00
#SBATCH --requeue
#SBATCH --mail-type=END
#SBATCH --mail-user=bjorn.larsen@yale.edu
#SBATCH --requeue

echo "SLURM_JOBID: " $SLURM_JOBID
echo "SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID
echo "SLURM_ARRAY_JOB_ID: " $SLURM_ARRAY_JOB_ID

module load miniconda
module load OpenMPI
conda activate PTA_env

export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export OPENBLAS_NUM_THREADS=1

gwb_Nfreqs=13
n_live=1000
n_networks=4
projectpath=/home/bbl29/IPTA_DR2_analysis
outdir=/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2lite_

In [30]:
datastr, datadir

('lite_unfiltered',
 '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis/data/lite_unfiltered_53_ePSRs')

In [31]:
for psrname in psrlist:
    print(psrname)
    # Write new file
    file_path = f'{basedir}/jobs/FL_{psrname}_{datastr}.sh'
    job_text = copy.deepcopy(template)
    job_text = job_text.replace("{{PSRNAME}}", psrname)
    job_text = job_text.replace("{{DATE}}", todays_date)
    job_text = job_text.replace("{{PSR}}", f'{datadir}/{psrname}.pkl')
    with open(file_path, "w") as f:
        f.write(job_text)

J0610-2100
J0900-3144
J1614-2230
J1721-2457
J1741+1351
J1824-2452A
J1903+0327
J1944+0907
J2019+2425
J2033+1734


In [24]:
basedir

'/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'

In [23]:
mass_submit_text = 'import os\n'
for psrname in psrlist:
    mass_submit_text += f'os.system(f"sbatch jobs/FL_{psrname}_{datastr}.sh")\n'
with open(f'{basedir}/submit_DR2{datastr}_factlike_10more.py', "w") as f:
    f.write(mass_submit_text)

In [25]:
datastr

'lite_unfiltered'

In [37]:
file_path

'/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis/jobs/FL_J2322+2057_full.sh'